<a href="https://colab.research.google.com/github/lapaniku/generative-sentiment/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers[torch] evaluate scikit-learn --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:

import urllib.request
urllib.request.urlretrieve("https://gist.githubusercontent.com/lapaniku/c45c3f0ae7b0481ea95e6c1086e4d3cb/raw/cb46a5361ff87e6c7ae0eaeec3a04aa779f5da84/regen_train.json", "regen.json")

('regen.json', <http.client.HTTPMessage at 0x7fea53bc5a50>)

In [3]:
#prepare dataset
import datasets
from datasets import Dataset, load_metric
import json

generated_tickets = json.load(open("regen.json"))

In [4]:
generated_tickets[:1]

[{'ticket': "Absolutely love this vacuum! It's lightweight and picks up everything on the first pass. The noise level is much lower than my old one, making cleaning less of a chore. Highly recommend.",
  'input': '',
  'output': 'Strong Positive',
  'most_similar_instructions': {"im using this card on my galaxy note 2 but it's slow, and phone constantly unable to read the card": 0.18518518518518517,
   "This is a 'set-it-and-forget-it' item. Solid performance without issues. I forgot it is still in my Sony camera, Love it": 0.1754385964912281,
   'Pictures were bright and sharp and download from the camera to the card very quickly. If you have a better quality camera this is a great card': 0.16666666666666666,
   'First of all Amazon has shipped the mobile version of this card. It is different that what was listed on the page. There goes a star for incorrect shipping. And the item has stopped working after a week in my Surface tablet.': 0.16,
   'I used this product with my Dell Venue 

In [5]:
tickets = [item["ticket"] for item in generated_tickets]
sentiments = [item["output"] for item in generated_tickets]
ds = Dataset.from_dict({"text": tickets, "labels": sentiments})

## Base model

https://huggingface.co/hyp1231/blair-roberta-base

BLaIR, which is short for "Bridging Language and Items for Retrieval and Recommendation", is a series of language models pre-trained on Amazon Reviews 2023 dataset.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("hyp1231/blair-roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Map:   0%|          | 0/1855 [00:00<?, ? examples/s]

In [7]:
from datasets import ClassLabel
ds = tokenized_datasets.cast_column("labels", ClassLabel(names = ["Strong Negative", "Mild Negative", "Neutral", "Mild Positive", "Strong Positive"] ))
class_label_feature = ds.features["labels"]
id2label = [ds.features['labels'].int2str(id) for id in range(5)]


Casting the dataset:   0%|          | 0/1855 [00:00<?, ? examples/s]

In [8]:

id2label

['Strong Negative',
 'Mild Negative',
 'Neutral',
 'Mild Positive',
 'Strong Positive']

In [9]:
ds = ds.train_test_split(test_size=0.3, stratify_by_column="labels")

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1298
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 557
    })
})

In [11]:
from transformers import AutoModelForSequenceClassification

# 110M parameters
model = AutoModelForSequenceClassification.from_pretrained("hyp1231/blair-roberta-base", num_labels=5)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at hyp1231/blair-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision_macro = precision_score(labels, predictions, average='macro', zero_division=0)
    recall_macro = recall_score(labels, predictions, average='macro', zero_division=0)
    precision_micro = precision_score(labels, predictions, average='micro', zero_division=0)
    recall_micro = recall_score(labels, predictions, average='micro', zero_division=0)
    conf_matrix = confusion_matrix(labels, predictions).tolist()

    return {
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro,
        'confusion_matrix': conf_matrix
    }

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", report_to="none")

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics,
)

From confusion matrix I see that the most challenging is to deal with negative part. For example strong negative and neutral are not clearly separated, also mild negative is noisy, but it could be just a lack of data and training.


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,Precision Micro,Recall Micro,Confusion Matrix
1,No log,0.735776,0.755596,0.758303,0.775583,0.775583,"[[54, 26, 0, 0, 0], [12, 115, 31, 6, 0], [0, 21, 52, 7, 0], [0, 1, 3, 54, 7], [0, 0, 0, 11, 157]]"
2,No log,0.936797,0.771593,0.733701,0.780969,0.780969,"[[67, 13, 0, 0, 0], [26, 130, 6, 2, 0], [0, 40, 33, 7, 0], [0, 4, 3, 43, 15], [0, 0, 0, 6, 162]]"
3,No log,0.968809,0.764252,0.757170,0.788151,0.788151,"[[62, 18, 0, 0, 0], [22, 123, 17, 2, 0], [0, 25, 51, 4, 0], [0, 1, 8, 44, 12], [0, 0, 0, 9, 159]]"


TrainOutput(global_step=489, training_loss=0.2642537372487698, metrics={'train_runtime': 458.0566, 'train_samples_per_second': 8.501, 'train_steps_per_second': 1.068, 'total_flos': 1024582046754816.0, 'train_loss': 0.2642537372487698, 'epoch': 3.0})

In [23]:

model_path = "generative-sentiment"
trainer.save_model(model_path)

In [24]:
!pip install huggingface_hub

In [25]:

from huggingface_hub import login
login()

In [26]:
from huggingface_hub import HfApi

api = HfApi()
model_repo_name = "alapanik/blair-roberta-base-generative-sentiment"
# api.create_repo(repo_id=model_repo_name)
api.upload_folder(
    folder_path=model_path,
    repo_id=model_repo_name
)
tokenizer.push_to_hub(model_repo_name)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alapanik/blair-roberta-base-generative-sentiment/commit/8574cce26b6c86593d8b27dae39d968296735644', commit_message='Upload tokenizer', commit_description='', oid='8574cce26b6c86593d8b27dae39d968296735644', pr_url=None, pr_revision=None, pr_num=None)